In [5]:
import os

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
from dotenv import load_dotenv

In [8]:
load_dotenv()

True

In [44]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
LANGCHAIN_TRACING_V2=os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"]=LANGCHAIN_TRACING_V2
os.environ["LANGCHAIN_API_KEY"]=LANGCHAIN_API_KEY
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY


tavily_api_key=os.getenv("tavily_api_key")
# os.environ["tavily_api_key"]=tavily_api_key
# LANGCHAIN_API_KEY
tavily_api_key

TypeError: str expected, not NoneType

### load the model and test it with the simple message

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [12]:
model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [13]:
message=[SystemMessage(content="Hi you are nice Bot"),HumanMessage(content="Hi, how are you buddy?")]

In [14]:
message2=[SystemMessage(content="Hi you are nice Bot and you answer everything in brief"),HumanMessage(content="Hi, how are you buddy?")]

In [15]:
model.invoke(message)

AIMessage(content='I am not BotHi, I am Gemini, a multi-modal AI language model developed by Google. But, I am happy to assist you with your questions and provide the best possible answers to the best of my abilities.\n\nMay I help you with something today?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-be9553d3-4ef2-4abb-b4d8-ae4879c96f3d-0', usage_metadata={'input_tokens': 13, 'output_tokens': 54, 'total_tokens': 67})

### use output parser

In [16]:
from langchain_core.output_parsers import StrOutputParser

In [17]:
parser = StrOutputParser()

In [18]:
response = model.invoke(message)

In [19]:
response2 = model.invoke(message2)

In [20]:
response2

AIMessage(content="I'm doing great, thanks!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-29358fe0-5754-48e2-90c1-707fa360a870-0', usage_metadata={'input_tokens': 19, 'output_tokens': 8, 'total_tokens': 27})

In [21]:
parser.invoke(response)

'I am well, thank you for asking. I am always happy to help in any way that I can. How may I assist you today?'

In [22]:
parser.invoke(response2)

"I'm doing great, thanks!"

### chaining with LCEL

In [23]:
#added two component
chain = model | parser

In [24]:
chain.invoke(message)

'I am well, thank you for asking. I am happy to be of assistance to you today. Is there anything I can help you with?'

In [25]:
chain.invoke(message2)

'Good, thanks!'

### Now lets start with prompting

In [27]:
from langchain_core.prompts import ChatPromptTemplate

In [28]:
system="translate the following into {language}"
#user="i am hungry."

In [29]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system","translate the following into {language}"),
     ("user","{text}")]
)

In [30]:
prompt = prompt_template.invoke({"language":"french","text":"i am learning langchain with sunny sir."})

In [31]:
parser.invoke(model.invoke(prompt))

"J'apprends LangChain avec Sunny Sir."

In [32]:
prompt.to_messages()

[SystemMessage(content='translate the following into french'),
 HumanMessage(content='i am learning langchain with sunny sir.')]

### chaining 

In [33]:
chain = prompt_template | model | parser

In [34]:
chain.invoke({"language": "hindi", "text": "I am hungry"})

'मुझे भूख लगी है'

### Agents

In [35]:

from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [36]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

In [37]:
memory = MemorySaver()

In [38]:
search=TavilySearchResults( max_results=2)

ValidationError: 1 validation error for TavilySearchAPIWrapper
__root__
  Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. (type=value_error)

In [32]:
tools=[search]

In [33]:
agent_executor=create_react_agent(model,tools,checkpointer=memory)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [34]:
config = {"configurable": {"thread_id": "abc123"}}

In [35]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="hi im sunny! and i live in bangalore")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="It's nice to meet you Sunny. How can I help you today?", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-309b4072-b381-4491-8df6-4afbbaa502f5-0', usage_metadata={'input_tokens': 84, 'output_tokens': 16, 'total_tokens': 100})]}}
----


In [36]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats the weather where I live?")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is the weather in Bangalore"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-9952e6bc-1006-4220-98a6-e82bf7a9970e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is the weather in Bangalore'}, 'id': '31999abb-6e79-45e7-968c-b19c15a6ba90', 'type': 'tool_call'}], usage_metadata={'input_tokens': 109, 'output_tokens': 25, 'total_tokens': 134})]}}
----
{'too